## 라이브러리 import

In [120]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import *
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
from eunjeon import Mecab

In [121]:
# dataframe print 이쁘게
pd.set_option('display.unicode.east_asian_width', True)

## 모델, 인코더, 토크나이저, 데이터 로드

In [122]:
# 모델 로드
model = load_model('./models/news_classification_0.8988.h5')

In [123]:
# Tokenizer 로드
with open('./data/nouns_token.pickle', 'rb') as f:
    token = pickle.load(f)

In [124]:
# LabelEncoder 로드
with open('./data/nouns_category_onehot_encoder.pickle', 'rb') as f:
    encoder = pickle.load(f)
category = encoder.categories_[0]
print(category)

['Art_Culture' 'Economic' 'Economic_Management' 'Health_Hobby_Leisure'
 'Home_Cook_Beauty' 'Novel_Poem' 'Reference_Book' 'Religion_Mysticism']


In [125]:
# 데이터 로드
df = pd.read_csv('./data/raw_test_446_2021-06-21.csv', index_col=0)
print(df.head())
print(df.info())

                                                                                                                          summary  \
title                                                                                                                               
글쓰기의 공중부양                           작가 이외수의 독특한 글쓰기 방법론이 담긴 책. 그동안 글쓰기를 배우려는 사람들을 ...     
행복한 사람은 시계를 보지 않는다            은희경의 두 번째 소설집. 작가는 인물에 대한 섬세한 심리 묘사와 속도감 있는 문체...      
두 번째 용사는 복수의 길을 웃으며 걷는다 7  현대에서 이세계로 돌아온 카이토는 우연히 마왕 레티시아와 재회했다. 하지만 레티시아...   
초판본 이상선집                             천재 소설가 이상의 유고 선집 <이상선집>이 1949년 백양당 오리지널 디자인 영인...         
하늘과 바람과 별과 시                       혜원출판사는 1977년부터 윤동주, 김소월, 한용운, 이육사, 김영랑 등 우리나라 ...          

                                            category  
title                                                 
글쓰기의 공중부양                           Economic  
행복한 사람은 시계를 보지 않는다            Economic  
두 번째 용사는 복수의 길을 웃으며 걷는다 7  Economic  
초판본 이상선집                             Economic  
하

## 전처리

### 중복값 제거

In [126]:
# 결측값 제거
df.reset_index(inplace=True)
df.drop_duplicates(subset=['title'], inplace=True)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
print(df['title'].duplicated().sum())

0


### feature, target 분리

In [127]:
X = df['title'] + " " + df['summary']
Y = df[['category']]

### 형태소 분석

In [128]:
# 데이터 형태소 분석
m = Mecab()
for i in range(len(X)):
    X[i] = m.nouns(X[i])
print(X)

0      [글쓰기, 공중부양, 작가, 이외수, 독특, 글쓰기, 방법, 책, 그동안, 글쓰기,...
1      [행복, 사람, 시계, 은희경, 번, 소설, 집, 작가, 인물, 심리, 묘사, 속도... 
2      [번, 용사, 복수, 길, 현대, 세계, 카이토, 마왕, 레티시아, 재회, 레티시아...
3      [초판본, 이상, 선집, 천재, 소설가, 이상, 유고, 선집, 이상선, 집, 년, ... 
4      [하늘, 바람, 별, 시, 혜원, 출판사, 년, 윤동주, 김소월, 한용운, 이육사,...
                                         ...                                    
374    [지식, 이중주, 시대, 중요, 가지, 키, 워드, 전공, 학자, 대화, 형식, 책... 
375    [퀴리, 마리, 퀴리, 그녀, 속, 퀴리, 가문, 서술, 마리, 퀴리, 외국인, 데... 
376    [광릉, 수목원, 사진, 일기, 광릉, 수목원, 생물체, 스케치, 사진, 세이, 일...
377    [지구, 재앙, 온난화, 현실, 전달, 알래스카, 주, 데드, 호스, 레이캬비크, ...
378    [기회, 확률, 법칙, 주식, 투자, 여론, 조사, 복권, 당첨, 배우자, 선택, ... 
Length: 379, dtype: object


In [129]:
# stopword 로드
stopwords = pd.read_csv('../datasets/stopwords.csv', index_col=0)
print(stopwords)

    stopword
0         아
1         휴
2     아이구
3     아이쿠
4     아이고
..       ...
783   없다는
784   않을까
785     있지
786   있으며
787   그대로

[787 rows x 1 columns]


In [130]:
# 함수로 만들어 기존 데이터에 apply해 stopword 제거
def delete_stopwords(lst):
    words = []
    for word in lst:
        if word not in list(stopwords['stopword']) and len(word) > 1:
            words.append(word)
    return ' '.join(words)

X = X.apply(delete_stopwords)

### 토큰화

In [131]:
# stopword 제거한 뉴스 타이틀 토큰화
tokened_X = token.texts_to_sequences(X)
print(tokened_X[0])

[729, 18996, 11, 18655, 741, 729, 5, 729, 241, 268, 35, 6194, 294, 1279, 325, 35, 6194, 268, 1455, 89, 729, 86, 6, 3383, 104, 14, 144, 780, 95, 7, 71, 9, 521, 868, 6467, 5, 145, 25, 740, 434, 434, 11629, 219, 5, 29, 69, 434, 5520, 5, 434, 2487, 1211, 5, 165, 239, 3134, 5, 6, 299, 299, 29, 260, 729, 315, 5267, 299, 935, 570, 7, 35, 299, 1039, 299, 7, 40916, 530, 782, 35, 13, 29, 298, 7, 654, 21, 712, 21, 18655, 299, 3068, 9]


### 문장 길이 맞춤

In [132]:
# 길이가 맞게 앞쪽에 0 붙여줌
max = 612
X_pad = pad_sequences(tokened_X, max)
print(X_pad[:10])

[[    0     0     0 ...   299  3068     9]
 [    0     0     0 ...  6249   617   300]
 [    0     0     0 ...  2130 19326  7061]
 ...
 [    0     0     0 ... 13247    16  4050]
 [    0     0     0 ...   650    48   173]
 [    0     0     0 ...  1653  3282   499]]


## 예측 결과 확인

In [133]:
category[0]

'Art_Culture'

In [134]:
predict = model.predict(X_pad)
print(predict[0])
print(category[np.argmax(predict[0])])

[3.7518700e-04 6.6136204e-06 2.5494082e-06 7.7695564e-05 7.9160136e-09
 9.9950123e-01 4.7173639e-06 3.1901342e-05]
Novel_Poem


In [135]:
 predict_category = [category[np.argmax(p)] for p in predict]
 print(predict_category)

['Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Art_Culture', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Economic', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Economic_Management', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Novel_Poem', 'Economic_Management', 'E

In [136]:
df['predict'] = predict_category

In [140]:
df['OX'] = df.apply(lambda x: 'O' if x['predict']==x['category'] else 'X', axis=1)

In [138]:
print(df['OX'].value_counts() / len(df['OX']))

X    0.881266
O    0.118734
Name: OX, dtype: float64
